In [75]:
## for data
import numpy as np
import pandas as pd
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for geospatial
import folium
import geopy
## for machine learning
from sklearn import preprocessing, cluster
import scipy
## for deep learning
import minisom

In [76]:
data = pd.read_csv("world_country_and_usa_states_latitude_and_longitude_values.csv")
data.head()

,country_code,latitude,longitude,country,usa_state_code,usa_state_latitude,usa_state_longitude,usa_state
0,AD,42.546245,1.601554,Andorra,AK,63.588753,-154.493062,Alaska
1,AE,23.424076,53.847818,United Arab Emirates,AL,32.318231,-86.902298,Alabama
2,AF,33.939110,67.709953,Afghanistan,AR,35.201050,-91.831833,Arkansas
3,AG,17.060816,-61.796428,Antigua and Barbuda,AZ,34.048928,-111.093731,Arizona
4,AI,18.220554,-63.068615,Anguilla,CA,36.778261,-119.417932,California


In [93]:
ww = data[['country_code', 'latitude', 'longitude', 'country']]
ww.head()

,country_code,latitude,longitude,country
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [94]:
ww.shape

(245, 4)

In [95]:
ww.tail(10)

,country_code,latitude,longitude,country
235,VN,14.058324,108.277199,Vietnam
236,VU,-15.376706,166.959158,Vanuatu
237,WF,-13.768752,-177.156097,Wallis and Futuna
238,WS,-13.759029,-172.104629,Samoa
239,XK,42.602636,20.902977,Kosovo
240,YE,15.552727,48.516388,Yemen
241,YT,-12.827500,45.166244,Mayotte
242,ZA,-30.559482,22.937506,South Africa
243,ZM,-13.133897,27.849332,Zambia
244,ZW,-19.015438,29.154857,Zimbabwe


In [96]:
ww.dropna(inplace=True)
ww.shape

<ipython-input-96-5c5f217eada1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ww.dropna(inplace=True)


(243, 4)

In [97]:
from k_means_constrained import KMeansConstrained

In [98]:
ww = ww.round(5)
ww.shape

(243, 4)

In [100]:
np.random.seed(1234)
# Run K-Means algorithm
neps = 10
capacity = 8
x = int(ww.shape[0]/capacity)
complement = int(capacity + ((50/100)*capacity))
clf = KMeansConstrained(
     n_clusters=x,
     size_min=capacity,
     size_max=complement,
     random_state=0
)

In [101]:
kmodel = clf.fit(ww[['latitude', 'longitude']])

In [102]:
# Assign labels into a column
ww["label"] = kmodel.labels_
ww.head()

,country_code,latitude,longitude,country,label
0,AD,42.54624,1.60155,Andorra,28
1,AE,23.42408,53.84782,United Arab Emirates,14
2,AF,33.93911,67.70995,Afghanistan,12
3,AG,17.06082,-61.79643,Antigua and Barbuda,13
4,AI,18.22055,-63.06862,Anguilla,13


In [103]:
ww['vlr_deb'] = np.random.randint(0,1000, size=len(ww))
ww.head(20)

,country_code,latitude,longitude,country,label,vlr_deb
0,AD,42.54624,1.60155,Andorra,28,815
1,AE,23.42408,53.84782,United Arab Emirates,14,723
2,AF,33.93911,67.70995,Afghanistan,12,294
3,AG,17.06082,-61.79643,Antigua and Barbuda,13,53
4,AI,18.22055,-63.06862,Anguilla,13,204
5,AL,41.15333,20.16833,Albania,21,372
6,AM,40.06910,45.03819,Armenia,8,664
7,AN,12.22608,-69.06009,Netherlands Antilles,2,655
8,AO,-11.20269,17.87389,Angola,4,689
9,AQ,-75.25097,-0.07139,Antarctica,19,279


In [104]:
def returnClusters(df, neps):
    df_aux = df.groupby(['label']).sum()
    df_aux['label'] = df_aux.index
    df_aux = df_aux[['label','vlr_deb']]
    df_aux.sort_values(by='vlr_deb', ascending=False, inplace=True)
    df_aux = df_aux.head(neps)
    clusters = df_aux['label'].unique()
    return clusters

clusters = returnClusters(ww, neps)
ww['alvo'] = np.where(ww['label'].isin(clusters), 1, 0)
ww.head(20)

,country_code,latitude,longitude,country,label,vlr_deb,alvo
0,AD,42.54624,1.60155,Andorra,28,815,1
1,AE,23.42408,53.84782,United Arab Emirates,14,723,0
2,AF,33.93911,67.70995,Afghanistan,12,294,0
3,AG,17.06082,-61.79643,Antigua and Barbuda,13,53,0
4,AI,18.22055,-63.06862,Anguilla,13,204,0
5,AL,41.15333,20.16833,Albania,21,372,0
6,AM,40.06910,45.03819,Armenia,8,664,0
7,AN,12.22608,-69.06009,Netherlands Antilles,2,655,1
8,AO,-11.20269,17.87389,Angola,4,689,1
9,AQ,-75.25097,-0.07139,Antarctica,19,279,0


In [105]:
ww.to_excel("ww_all_countries.xlsx")

In [106]:
ww = ww[ww['alvo']==1]
ww.head(10)

,country_code,latitude,longitude,country,label,vlr_deb,alvo
0,AD,42.54624,1.60155,Andorra,28,815,1
7,AN,12.22608,-69.06009,Netherlands Antilles,2,655,1
8,AO,-11.20269,17.87389,Angola,4,689,1
13,AU,-25.27440,133.77514,Australia,0,683,1
14,AW,12.52111,-69.96834,Aruba,2,158,1
21,BG,42.73388,25.48583,Bulgaria,3,880,1
23,BI,-3.37306,29.91889,Burundi,4,325,1
25,BM,32.32138,-64.75737,Bermuda,25,969,1
26,BN,4.53528,114.72767,Brunei,0,383,1
29,BS,25.03428,-77.39628,Bahamas,25,431,1


In [107]:
ww.sort_values(by=['label', 'vlr_deb'], ascending=False, inplace=True)
ww['util'] = ww.groupby('label')['vlr_deb'].rank(ascending=False)
ww.head(8)

,country_code,latitude,longitude,country,label,vlr_deb,alvo,util
71,FR,46.22764,2.21375,France,28,997,1,1.0
134,MA,31.79170,-7.09262,Morocco,28,967,1,2.0
79,GI,36.13774,-5.34537,Gibraltar,28,840,1,3.0
0,AD,42.54624,1.60155,Andorra,28,815,1,4.0
64,ES,40.46367,-3.74922,Spain,28,781,1,5.0
58,DZ,28.03389,1.65963,Algeria,28,490,1,6.0
180,PT,39.39987,-8.22445,Portugal,28,46,1,7.0
108,JE,49.21444,-2.13125,Jersey,28,10,1,8.0


In [108]:
def define_alvo(df, capacity):
    df['util'] = np.where(df['util'] <= capacity, 'ALVO', 'COMPLEMENTO')
    return df

ww = define_alvo(ww, capacity)
ww.head(8)

,country_code,latitude,longitude,country,label,vlr_deb,alvo,util
71,FR,46.22764,2.21375,France,28,997,1,ALVO
134,MA,31.79170,-7.09262,Morocco,28,967,1,ALVO
79,GI,36.13774,-5.34537,Gibraltar,28,840,1,ALVO
0,AD,42.54624,1.60155,Andorra,28,815,1,ALVO
64,ES,40.46367,-3.74922,Spain,28,781,1,ALVO
58,DZ,28.03389,1.65963,Algeria,28,490,1,ALVO
180,PT,39.39987,-8.22445,Portugal,28,46,1,ALVO
108,JE,49.21444,-2.13125,Jersey,28,10,1,ALVO


In [109]:
ww['equipe'] = ww['country'].map(str) + '_' + ww['label'].map(str)
ww.head(100)

,country_code,latitude,longitude,country,label,vlr_deb,alvo,util,equipe
71,FR,46.22764,2.21375,France,28,997,1,ALVO,France_28
134,MA,31.79170,-7.09262,Morocco,28,967,1,ALVO,Morocco_28
79,GI,36.13774,-5.34537,Gibraltar,28,840,1,ALVO,Gibraltar_28
0,AD,42.54624,1.60155,Andorra,28,815,1,ALVO,Andorra_28
64,ES,40.46367,-3.74922,Spain,28,781,1,ALVO,Spain_28
58,DZ,28.03389,1.65963,Algeria,28,490,1,ALVO,Algeria_28
180,PT,39.39987,-8.22445,Portugal,28,46,1,ALVO,Portugal_28
108,JE,49.21444,-2.13125,Jersey,28,10,1,ALVO,Jersey_28
25,BM,32.32138,-64.75737,Bermuda,25,969,1,ALVO,Bermuda_25
48,CU,21.52176,-77.78117,Cuba,25,908,1,ALVO,Cuba_25


In [111]:
pd.set_option('display.max_rows', None)
ww.to_excel("world_countries_alvos.xlsx")